In [13]:
import json
# data = json.load(open('output_dialog_missing_1.json', 'r'))
data = json.load(open('output_dialog_redial.json', 'r'))

In [3]:
# Get the mentioned movies
# with multiple movies
list(data[0].keys())[0].split(' ')
# with individual movies
list(data[0].keys())[0]

'<http://dbpedia.org/resource/The_Thin_Red_Line_(1998_film)>'

In [15]:
entity2id = json.load(open('redial_nltk/entity2id.json', 'r'))

In [16]:
entity2id[list(data[0].keys())[0]]

3

In [7]:
import re
# Get the utterances
# list(data[0].values())[0]['choices'][0]['message']['content'].split('\n')
re.split(r'\n\n|\n', list(data[0].values())[0])

["[user]: Hey, I'm in the mood for a movie. Do you have any recommendations? ",
 "[system]: Of course! How about The Thin Red Line? It's a 1998 war film directed by Terrence Malick that explores the World War II Battle of Guadalcanal through the perspective of various soldiers. ",
 "[user]: Hmm, I'm not really into war movies, are there any other genres you recommend? ",
 "[system]: Sure thing! If you're looking for something in the same vein as The Thin Red Line, I would recommend Saving Private Ryan. If you're looking for something different, Though, I'd suggest perhaps a classic comedy like The Pink Panther. ",
 '[user]: I think The Pink Panther sounds good. Is it a silly comedy or more of a dark humor kind of thing? ',
 "[system]: The Pink Panther is definitely more of a silly comedy! It's a classic film from the sixties that follows Inspector Clouseau, played by Peter Sellers, on his hilarious attempt to catch a jewel thief. ",
 '[user]: Sounds like a great pick-me-up. Thanks for 

### Utterance Process

In [18]:
from fuzzywuzzy import fuzz
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import string

def fuzzy_match(uttr, movie_name):
    # print(fuzz.partial_ratio(uttr, movie_name))
    if fuzz.partial_ratio(uttr, movie_name) > 60:
        return True
    else:
        return False

def fuzzy_replace(str_a, str_b, orig_str):
    l = len(str_a.split()) # Length to read orig_str chunk by chunk
    splitted = orig_str.split()
    for i in range(len(splitted)-l+1):
        test = " ".join(splitted[i:i+l])
        if fuzz.partial_ratio(str_a, test) > 60: #Using fuzzwuzzy library to test ratio
            before = " ".join(splitted[:i])
            after = " ".join(splitted[min(len(splitted), i+l):])
            return before+" "+str_b+" "+after #Output will be sandwich of these three strings


def utterance_convert(uttr, movies, movie_ids):
    uttr = uttr.lower()
    new_uttr = ''
    for char in uttr:
        if char not in string.punctuation:
            new_uttr += char
    uttr = new_uttr
    movie_mentioned = []
    for movie in movies:
        if fuzzy_match(uttr, ' '.join(movie[29:-1].split('_'))):
            uttr = fuzzy_replace(' '.join(movie[29:-1].split('_')), "@" + str(movie_ids[movie]), uttr)
            movie_mentioned.append(movie)
            break
    if uttr is None:
        text = []
    else:
        text = uttr.split(' ')
    filtered_words = [word for word in text if word not in stopwords.words('english')]
    return text, filtered_words, movie_mentioned

# # Test instance
# movies = ['<http://dbpedia.org/resource/Her_Battle_for_Existence>', '<http://dbpedia.org/resource/12_Rounds_(film)>','<http://dbpedia.org/resource/The_Three_Musketeers_(1986_film)>']
# movie_ids = {'<http://dbpedia.org/resource/Her_Battle_for_Existence>':101245, '<http://dbpedia.org/resource/12_Rounds_(film)>':123, '<http://dbpedia.org/resource/The_Three_Musketeers_(1986_film)>':456}
# uttr1 = "What kind of movies are you in the mood for?"
# uttr2 = "Sure, how about 'Her Battle for Existence?' It\'s a documentary about a woman\'s journey to overcome a rare and life-threatening disease.'"
# uttr3 = "You might enjoy '12 Rounds,' which is an action-packed thriller about a cop trying to save his kidnapped girlfriend."
# uttr4 = "Have you seen 'The Three Musketeers' from 1986? It\'s an adventure movie set in 17th century France about three musketeers who protect the king."
# utterance_convert(uttr4, movies, movie_ids)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
import pandas as pd
from fuzzywuzzy import fuzz
import tqdm
from datetime import datetime
import re
from collections import defaultdict

def cal_sim(name):
    return fuzz.ratio(movie, name)

def name_to_link(name, movie_ids):
    movie_ids['sim'] = movie_ids['movieName'].apply(lambda x: fuzz.ratio(name, x))
    return movie_ids[movie_ids['sim'] == movie_ids['sim'].max()]

movie_ids = pd.read_csv('redial_nltk/movies_with_mentions.csv')
syn_data = defaultdict(dict)

for conv_idx, instance in enumerate(data):
    if conv_idx % 100 == 0:
        print("progress: ", conv_idx / len(data) * 100, "%", datetime.now())
    syn_data_inst = {}
    movie2id = {}
    potential_mentioned_movies = list(instance.keys())[0].split(' ')
    for movie in potential_mentioned_movies:
        movie2id[movie] = list(name_to_link(' '.join(movie[29:-1].split('_')), movie_ids)['movieId'])[0]
    
    # assign conversation id     
    conv_id = 'syn_' + str(conv_idx)
    syn_data_inst['conv_id'] = conv_id
    
    syn_data_inst['dialog'] = []
    # utterances = list(instance.values())[0]['choices'][0]['message']['content'].split('\n')
    utterances = re.split(r'\n\n|\n', list(data[0].values())[0])
    utterances = list(filter(None, utterances))
    
    broken_dialog = False
    for idx, utterance in enumerate(utterances):
        uttr_info = {}
        utt_id = idx
        if utterance[:6].lower() == '[user]':
            role = 'Seeker'
            text, filtered_words, movie_mentioned = utterance_convert(utterance[7:], potential_mentioned_movies, movie2id)
        elif utterance[:8].lower() == '[system]':
            role = 'Recommender'
            text, filtered_words, movie_mentioned = utterance_convert(utterance[9:], potential_mentioned_movies, movie2id)
        else:
            broken_dialog = True
        if broken_dialog:
            break
        else:
            uttr_info['uttr_id'] = idx
            uttr_info['role'] = role
            uttr_info['text'] = text
            uttr_info['word'] = filtered_words
            uttr_info['entity'] = []
            uttr_info['movies'] = movie_mentioned
            syn_data_inst['dialog'].append(uttr_info)
    if broken_dialog:
        continue
    syn_data[str(entity2id[list(instance.keys())[0]])] = syn_data_inst

progress:  0.0 % 2023-06-14 09:22:21.268047
progress:  1.6361256544502618 % 2023-06-14 09:23:00.377913
progress:  3.2722513089005236 % 2023-06-14 09:23:38.827619
progress:  4.908376963350785 % 2023-06-14 09:24:17.280930
progress:  6.544502617801047 % 2023-06-14 09:24:56.635441
progress:  8.180628272251308 % 2023-06-14 09:25:34.006651
progress:  9.81675392670157 % 2023-06-14 09:26:13.130576
progress:  11.452879581151834 % 2023-06-14 09:26:51.927454
progress:  13.089005235602095 % 2023-06-14 09:27:29.479976
progress:  14.725130890052357 % 2023-06-14 09:28:08.466045
progress:  16.361256544502616 % 2023-06-14 09:28:45.841109
progress:  17.99738219895288 % 2023-06-14 09:29:23.564729
progress:  19.63350785340314 % 2023-06-14 09:30:02.544655
progress:  21.269633507853403 % 2023-06-14 09:30:41.327822
progress:  22.905759162303667 % 2023-06-14 09:31:19.686817
progress:  24.541884816753925 % 2023-06-14 09:31:58.122173
progress:  26.17801047120419 % 2023-06-14 09:32:36.376897
progress:  27.814136

In [23]:
with open('output_dialog_processed_redial_dict.json', 'w') as outfile:
    json.dump(syn_data, outfile)

In [10]:
with open('output_dialog_processed_redial.json', 'w') as outfile:
    json.dump(syn_data, outfile)

In [4]:
with open('output_dialog_processed_2.json', 'w') as outfile:
    json.dump(syn_data, outfile)

In [6]:
data = json.load(open('output_dialog_processed.json', 'r'))

In [7]:
data.extend(syn_data)

In [9]:
with open('output_dialog_processed_normal.json', 'w') as outfile:
    json.dump(data, outfile)